
# ANALITIKA DATA: EXPLORATORY DATA ANALYSIS

disusun oleh:
- Dr.Eng.Farrikh Alzami, M.Kom
- Ika Novita Dewi, MCs

**Universitas Dian Nuswantoro**

#Bank Marketing Campaign Dataset

**Mata Kuliah**: Analitika Data

**Dataset**: Bank Marketing (UCI ML Repository)

**Tujuan**: Memahami faktor-faktor yang mempengaruhi keputusan nasabah untuk subscribe term deposit

**keterangan**: pakai yang full-additional

**link dataset**: https://archive.ics.uci.edu/dataset/222/bank+marketing


---
##📚 Learning Outcomes:
1. Mampu melakukan eksplorasi dan visualisasi data secara deskriptif
2. Mampu melakukan pembersihan dan transformasi data
3. Mampu menyusun insight dan storytelling berbasis data
4. Mempersiapkan data untuk pemodelan machine learning

---
##📁 Struktur Tutorial:
- **PERTEMUAN 1**: Foundation (Setup, Loading, Cleaning, Basic Stats)
- **PERTEMUAN 2**: Deep Exploration (Univariate, Bivariate, Multivariate)
- **PERTEMUAN 3**: Insights & Preparation (Feature Engineering, Storytelling, Export)



# # SETUP & INITIALIZATION

# Import libraries

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import os
from datetime import datetime

# Konfigurasi
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Set display options untuk pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# load google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')

In [ ]:

BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/0 1 analitika data/'  # Path base untuk Drive
FIGURES_PATH = BASE_PATH + 'figures/'

In [ ]:
# Create directory untuk menyimpan figures
if not os.path.exists(FIGURES_PATH):
    os.makedirs(FIGURES_PATH)

print("Setup completed successfully!")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Save path: {FIGURES_PATH}")

In [ ]:
# ====================================================================
# HELPER FUNCTIONS
# ====================================================================

def save_figure(filename):
    """Helper function to save figures with correct path"""
    plt.savefig(FIGURES_PATH + filename, dpi=300, bbox_inches='tight')
    print(f"Saved: {filename}")

def save_csv(df, filename):
    """Helper function to save CSV with correct path"""
    df.to_csv(FIGURES_PATH + filename, index=False)
    print(f"Saved: {filename}")

def save_text(text, filename):
    """Helper function to save text file with correct path"""
    with open(FIGURES_PATH + filename, 'w') as f:
        f.write(text)
    print(f"Saved: {filename}")

#1.1 DATA LOADING & OVERVIEW

Dataset Bank Marketing berisi informasi tentang kampanye marketing langsung dari institusi perbankan Portugal.

**Konteks Bisnis**:
- Bank ingin meningkatkan jumlah nasabah yang subscribe term deposit
- Marketing dilakukan via telepon
- Perlu identifikasi karakteristik nasabah yang potensial

**Download Dataset**:
- Source: https://archive.ics.uci.edu/ml/datasets/Bank+Marketing
- Atau Kaggle: https://www.kaggle.com/datasets/henriqueyamahata/bank-marketing
- File: bank-additional-full.csv (atau bank.csv)

In [ ]:
path = BASE_PATH + 'bank-additional-full.csv'

In [ ]:
df = pd.read_csv(path, sep=';')

In [ ]:
# Quick overview
print("="*70)
print("DATASET OVERVIEW")
print("="*70)
print(f"\nDimensi Dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\n" + "="*70)

# Tampilkan 5 baris pertama
df.head()

In [ ]:
# Informasi struktur data
print("\nINFORMASI KOLOM:")
print("="*70)
df.info()

##Penjelasan Variabel

**Variabel Input - Data Client:**

1. `age`: Usia nasabah (numerik)
2. `job`: Jenis pekerjaan (kategorikal)
3. `marital`: Status pernikahan (kategorikal)
4. `education`: Tingkat pendidikan (kategorikal)
5. `default`: Apakah memiliki kredit default? (kategorikal)
6. `housing`: Apakah memiliki kredit rumah? (kategorikal)
7. `loan`: Apakah memiliki kredit personal? (kategorikal)

**Variabel Input - Kampanye Saat Ini:**

8. `contact`: Tipe komunikasi kontak (kategorikal)
9. `month`: Bulan kontak terakhir (kategorikal)
10. `day_of_week`: Hari kontak terakhir (kategorikal)
11. `duration`: Durasi kontak terakhir dalam detik (numerik)

**Variabel Input - Atribut Lain:**

12. `campaign`: Jumlah kontak selama kampanye ini (numerik)
13. `pdays`: Jumlah hari sejak kontak terakhir dari kampanye sebelumnya (numerik)
14. `previous`: Jumlah kontak sebelum kampanye ini (numerik)
15. `poutcome`: Outcome dari kampanye marketing sebelumnya (kategorikal)

**Variabel Input - Sosial & Ekonomi:**

16. `emp.var.rate`: Employment variation rate (numerik)
17. `cons.price.idx`: Consumer price index (numerik)
18. `cons.conf.idx`: Consumer confidence index (numerik)
19. `euribor3m`: Euribor 3 month rate (numerik)
20. `nr.employed`: Number of employees (numerik)

**Variabel Output (Target):**

21. `y`: Apakah client subscribe term deposit? (binary: yes/no)

#1.2 DATA QUALITY ASSESSMENT

Sebelum analisis, kita perlu memastikan kualitas data:
- Missing values
- Duplicate rows
- Data types
- Outliers potensial
- Konsistensi data


In [ ]:
# Cek missing values
print("\n🔍 MISSING VALUES ANALYSIS:")
print("="*70)

missing = df.isnull().sum()
missing_pct = 100 * missing / len(df)
missing_table = pd.DataFrame({
    'Missing_Count': missing,
    'Percentage': missing_pct
})
missing_table = missing_table[missing_table['Missing_Count'] > 0].sort_values(
    'Percentage', ascending=False
)

if len(missing_table) > 0:
    print(missing_table)
else:
    print("Tidak ada missing values!")

In [ ]:
# Cek duplicate rows
duplicates = df.duplicated().sum()
print(f"\nDUPLICATE ROWS: {duplicates}")

if duplicates > 0:
    print(f"Ditemukan {duplicates} baris duplikat ({100*duplicates/len(df):.2f}%)")
else:
    print("Tidak ada baris duplikat!")


In [ ]:
# Cek tipe data dan unique values
print("\nDATA TYPES & UNIQUE VALUES:")
print("="*70)

data_overview = pd.DataFrame({
    'Column': df.columns,
    'Type': df.dtypes,
    'Unique': [df[col].nunique() for col in df.columns],
    'Sample_Values': [df[col].unique()[:3] for col in df.columns]
})

print(data_overview.to_string(index=False))


In [ ]:
# Pisahkan variabel numerik dan kategorikal
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Remove target dari list jika ada
if 'y' in numerical_cols:
    numerical_cols.remove('y')
if 'y' in categorical_cols:
    categorical_cols.remove('y')

print(f"\nVariabel Numerik ({len(numerical_cols)}): {numerical_cols}")
print(f"Variabel Kategorikal ({len(categorical_cols)}): {categorical_cols}")
print(f"Target Variable: y")

#1.3 DATA CLEANING & TRANSFORMATION

**Konsep Penting:**
- **Unknown values**: Beberapa kolom kategorikal punya nilai "unknown"
- **Pdays = 999**: Artinya client tidak pernah dikontak sebelumnya
- **Data consistency**: Pastikan format data konsisten


In [ ]:
# Cek nilai "unknown" di kolom kategorikal
print("\n'UNKNOWN' VALUES IN CATEGORICAL COLUMNS:")
print("="*70)

for col in categorical_cols:
    unknown_count = (df[col] == 'unknown').sum()
    if unknown_count > 0:
        unknown_pct = 100 * unknown_count / len(df)
        print(f"{col:20s}: {unknown_count:6d} ({unknown_pct:5.2f}%)")

In [ ]:
# Handle pdays: 999 berarti tidak pernah dikontak
# Kita buat feature baru yang lebih interpretable
df['was_contacted_before'] = (df['pdays'] != 999).astype(int)

print("\nFeature engineering: 'was_contacted_before' created")
print(f"   - Was contacted before: {df['was_contacted_before'].sum()} ({100*df['was_contacted_before'].mean():.2f}%)")
print(f"   - Never contacted: {(~df['was_contacted_before'].astype(bool)).sum()} ({100*(1-df['was_contacted_before'].mean()):.2f}%)")


In [ ]:
# Create backup of original data
df_original = df.copy()
print("\nBackup data original disimpan di 'df_original'")


#1.4 DESCRIPTIVE STATISTICS

**Statistik Deskriptif** memberikan ringkasan tentang pusat dan penyebaran data.

**Key Metrics:**
- **Mean (rata-rata)**: Nilai tengah, sensitif terhadap outlier
- **Median**: Nilai tengah yang membagi data jadi 2 bagian sama
- **Std (standar deviasi)**: Ukuran penyebaran data
- **Min/Max**: Nilai minimum dan maksimum
- **Quartiles (25%, 50%, 75%)**: Membagi data jadi 4 bagian sama


In [ ]:
# Descriptive statistics untuk variabel numerik
print("\nDESCRIPTIVE STATISTICS - NUMERICAL VARIABLES:")
print("="*70)

desc_stats = df[numerical_cols].describe().T
desc_stats['skewness'] = df[numerical_cols].skew()
desc_stats['kurtosis'] = df[numerical_cols].kurtosis()

print(desc_stats)

# Save to CSV for report
desc_stats.to_csv(FIGURES_PATH + 'descriptive_stats_numerical.csv')
print("\nSaved: descriptive_stats_numerical.csv")


## Interpretasi Skewness & Kurtosis:

**Skewness (Kemiringan)**:
- **Skew = 0**: Distribusi simetris (normal)
- **Skew > 0**: Right-skewed (ekor panjang ke kanan), lebih banyak nilai kecil
- **Skew < 0**: Left-skewed (ekor panjang ke kiri), lebih banyak nilai besar
- **|Skew| > 1**: Highly skewed

**Kurtosis (Keruncingan)**:
- **Kurt = 0**: Normal distribution
- **Kurt > 0**: Heavy tails (lebih banyak outliers)
- **Kurt < 0**: Light tails (lebih sedikit outliers)


In [ ]:
# Descriptive statistics untuk variabel kategorikal
print("\nDESCRIPTIVE STATISTICS - CATEGORICAL VARIABLES:")
print("="*70)

cat_summary = []
for col in categorical_cols:
    unique_count = df[col].nunique()
    most_common = df[col].mode()[0]
    most_common_pct = 100 * (df[col] == most_common).sum() / len(df)

    cat_summary.append({
        'Variable': col,
        'Unique_Values': unique_count,
        'Most_Common': most_common,
        'Most_Common_Pct': f"{most_common_pct:.2f}%"
    })

cat_summary_df = pd.DataFrame(cat_summary)
print(cat_summary_df.to_string(index=False))

# Save to CSV
cat_summary_df.to_csv(FIGURES_PATH + 'descriptive_stats_categorical.csv', index=False)
print("\nSaved: descriptive_stats_categorical.csv")


In [ ]:
# Target variable distribution
print("\nTARGET VARIABLE DISTRIBUTION:")
print("="*70)

target_dist = df['y'].value_counts()
target_dist_pct = 100 * df['y'].value_counts(normalize=True)

target_summary = pd.DataFrame({
    'Class': target_dist.index,
    'Count': target_dist.values,
    'Percentage': target_dist_pct.values
})

print(target_summary.to_string(index=False))

# Visualize target distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Count plot
target_dist.plot(kind='bar', ax=axes[0], color=['#FF6B6B', '#4ECDC4'])
axes[0].set_title('Target Variable Distribution (Count)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Subscribed Term Deposit?')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['No', 'Yes'], rotation=0)

# Add value labels on bars
for i, v in enumerate(target_dist.values):
    axes[0].text(i, v + 500, str(v), ha='center', va='bottom', fontweight='bold')

# Percentage plot
target_dist_pct.plot(kind='bar', ax=axes[1], color=['#FF6B6B', '#4ECDC4'])
axes[1].set_title('Target Variable Distribution (Percentage)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Subscribed Term Deposit?')
axes[1].set_ylabel('Percentage (%)')
axes[1].set_xticklabels(['No', 'Yes'], rotation=0)

# Add percentage labels
for i, v in enumerate(target_dist_pct.values):
    axes[1].text(i, v + 1, f'{v:.1f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig(FIGURES_PATH + '01_target_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nSaved: 01_target_distribution.png")


##💡 Key Insight #1: Class Imbalance

Dataset ini memiliki **class imbalance** yang signifikan. Mayoritas nasabah tidak subscribe term deposit.
Ini adalah **masalah klasifikasi yang imbalanced**, perlu diperhatikan saat modeling nanti.


---
🔬 PERTEMUAN 2: DEEP EXPLORATION
---

#2.1 UNIVARIATE ANALYSIS - NUMERICAL VARIABLES

**Univariate Analysis** = analisis satu variabel pada satu waktu.

**Tujuan:**
- Memahami distribusi setiap variabel
- Identifikasi outliers
- Memahami range dan variabilitas data


In [ ]:
# Distribusi variabel numerik dengan histogram
fig, axes = plt.subplots(5, 3, figsize=(18, 20))
axes = axes.ravel()

for idx, col in enumerate(numerical_cols):
    # Histogram dengan KDE (Kernel Density Estimation)
    axes[idx].hist(df[col], bins=50, edgecolor='black', alpha=0.7, color='skyblue')
    axes[idx].axvline(df[col].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df[col].mean():.2f}')
    axes[idx].axvline(df[col].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df[col].median():.2f}')

    axes[idx].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frequency')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

# Remove empty subplots
for idx in range(len(numerical_cols), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.savefig(FIGURES_PATH + '02_numerical_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved: 02_numerical_distributions.png")

## Cara Membaca Distribusi:

1. **Bentuk distribusi**:
   - **Normal/Bell-shaped**: Data terpusat di sekitar mean
   - **Right-skewed**: Banyak nilai kecil, few extreme large values
   - **Left-skewed**: Banyak nilai besar, few extreme small values
   - **Bimodal**: Ada 2 puncak (mungkin 2 grup berbeda)

2. **Mean vs Median**:
   - Jika **Mean ≈ Median**: Distribusi simetris
   - Jika **Mean > Median**: Right-skewed (outlier besar tarik mean ke kanan)
   - Jika **Mean < Median**: Left-skewed

In [ ]:
# Box plots untuk deteksi outliers
fig, axes = plt.subplots(5, 3, figsize=(18, 20))
axes = axes.ravel()

for idx, col in enumerate(numerical_cols):
    bp = axes[idx].boxplot(df[col], vert=True, patch_artist=True)

    # Customize box plot
    for patch in bp['boxes']:
        patch.set_facecolor('lightblue')
        patch.set_alpha(0.7)

    axes[idx].set_title(f'Box Plot: {col}', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel(col)
    axes[idx].grid(True, alpha=0.3)

    # Add statistics text
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()

    axes[idx].text(0.02, 0.98, f'Outliers: {outliers}',
                   transform=axes[idx].transAxes, fontsize=10,
                   verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Remove empty subplots
for idx in range(len(numerical_cols), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.savefig(FIGURES_PATH + '03_boxplots_outliers.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved: 03_boxplots_outliers.png")

## Cara Membaca Box Plot:

**Komponen Box Plot**:
- **Box (kotak)**: Berisi 50% data tengah (Q1 to Q3)
- **Line in box**: Median (Q2)
- **Whiskers (garis)**: Extend sampai 1.5×IQR dari Q1 dan Q3
- **Dots/Points**: Outliers (nilai ekstrem di luar whiskers)

**IQR (Interquartile Range) = Q3 - Q1**

**Outlier Detection Rule**:
- Lower bound = Q1 - 1.5×IQR
- Upper bound = Q3 + 1.5×IQR
- Any value outside bounds = outlier

In [ ]:
# Hitung outliers untuk setiap variabel numerik
outlier_summary = []

for col in numerical_cols:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    outliers = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
    outliers_pct = 100 * outliers / len(df)

    outlier_summary.append({
        'Variable': col,
        'Q1': q1,
        'Q3': q3,
        'IQR': iqr,
        'Lower_Bound': lower_bound,
        'Upper_Bound': upper_bound,
        'Outliers_Count': outliers,
        'Outliers_Pct': f"{outliers_pct:.2f}%"
    })

outlier_df = pd.DataFrame(outlier_summary)
print("\nOUTLIER DETECTION SUMMARY:")
print("="*70)
print(outlier_df.to_string(index=False))

# Save to CSV
outlier_df.to_csv(FIGURES_PATH + 'outlier_summary.csv', index=False)
print("\nSaved: outlier_summary.csv")

#2.2 UNIVARIATE ANALYSIS - CATEGORICAL VARIABLES

Untuk variabel kategorikal, kita lihat:
- Distribusi frekuensi setiap kategori
- Kategori mana yang dominan
- Apakah ada kategori yang jarang muncul


In [ ]:
# Bar plots untuk variabel kategorikal
n_categorical = len(categorical_cols)
n_rows = (n_categorical + 2) // 3
fig, axes = plt.subplots(n_rows, 3, figsize=(18, n_rows * 4))
axes = axes.ravel()

for idx, col in enumerate(categorical_cols):
    value_counts = df[col].value_counts()

    # Create bar plot
    value_counts.plot(kind='bar', ax=axes[idx], color='steelblue', edgecolor='black')
    axes[idx].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Count')
    axes[idx].tick_params(axis='x', rotation=45)
    axes[idx].grid(True, alpha=0.3, axis='y')

    # Add percentage labels on bars
    total = len(df)
    for i, v in enumerate(value_counts.values):
        pct = 100 * v / total
        axes[idx].text(i, v + total*0.01, f'{pct:.1f}%', ha='center', va='bottom', fontsize=9)

# Remove empty subplots
for idx in range(len(categorical_cols), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.savefig(FIGURES_PATH + '04_categorical_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved: 04_categorical_distributions.png")

In [ ]:
# Detailed frequency table untuk variabel kategorikal
print("\nFREQUENCY TABLES - CATEGORICAL VARIABLES:")
print("="*70)

for col in categorical_cols:
    print(f"\n{col.upper()}:")
    print("-" * 50)

    freq_table = pd.DataFrame({
        'Category': df[col].value_counts().index,
        'Count': df[col].value_counts().values,
        'Percentage': df[col].value_counts(normalize=True).values * 100
    })

    print(freq_table.to_string(index=False))

    # Save each frequency table
    freq_table.to_csv(FIGURES_PATH + f'freq_table_{col}.csv', index=False)

print(f"\nAll frequency tables saved to '{FIGURES_PATH}'")

#2.3 BIVARIATE ANALYSIS - Numerical vs Target

**Bivariate Analysis** = analisis hubungan antara 2 variabel.

Di sini kita lihat bagaimana variabel numerik berbeda antara:
- Nasabah yang subscribe (y=yes)
- Nasabah yang tidak subscribe (y=no)

In [ ]:
# Comparison box plots: Numerical variables vs Target
n_numerical = len(numerical_cols)
n_rows = (n_numerical + 2) // 3
fig, axes = plt.subplots(n_rows, 3, figsize=(18, n_rows * 4))
axes = axes.ravel()

for idx, col in enumerate(numerical_cols):
    # Prepare data for each class
    data_no = df[df['y'] == 'no'][col]
    data_yes = df[df['y'] == 'yes'][col]

    # Create box plot
    bp = axes[idx].boxplot([data_no, data_yes],
                           labels=['No', 'Yes'],
                           patch_artist=True,
                           showmeans=True)

    # Customize colors
    colors = ['#FF6B6B', '#4ECDC4']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)

    axes[idx].set_title(f'{col} vs Target', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Subscribed?')
    axes[idx].set_ylabel(col)
    axes[idx].grid(True, alpha=0.3)

    # Add mean values as text
    mean_no = data_no.mean()
    mean_yes = data_yes.mean()
    axes[idx].text(0.02, 0.98, f'Mean (No): {mean_no:.2f}\nMean (Yes): {mean_yes:.2f}',
                   transform=axes[idx].transAxes, fontsize=9,
                   verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Remove empty subplots
for idx in range(len(numerical_cols), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.savefig(FIGURES_PATH + '05_numerical_vs_target.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved: 05_numerical_vs_target.png")

In [ ]:
# Statistical tests: T-test untuk membandingkan means
# H0: Mean kedua grup sama
# H1: Mean kedua grup berbeda
# p-value < 0.05 → reject H0 (ada perbedaan signifikan)

print("\nT-TEST RESULTS: Numerical Variables vs Target")
print("="*70)
print("H0: Tidak ada perbedaan mean antara yes dan no")
print("H1: Ada perbedaan mean yang signifikan")
print("Significance level (α): 0.05")
print("="*70)

ttest_results = []

for col in numerical_cols:
    data_no = df[df['y'] == 'no'][col]
    data_yes = df[df['y'] == 'yes'][col]

    # Perform t-test
    t_stat, p_value = stats.ttest_ind(data_no, data_yes)

    # Calculate effect size (Cohen's d)
    mean_diff = data_yes.mean() - data_no.mean()
    pooled_std = np.sqrt((data_no.std()**2 + data_yes.std()**2) / 2)
    cohens_d = mean_diff / pooled_std

    # Interpret results
    is_significant = "Yes" if p_value < 0.05 else "No"

    ttest_results.append({
        'Variable': col,
        'Mean_No': data_no.mean(),
        'Mean_Yes': data_yes.mean(),
        'Mean_Diff': mean_diff,
        'T_Statistic': t_stat,
        'P_Value': p_value,
        'Significant': is_significant,
        'Cohens_d': cohens_d
    })

ttest_df = pd.DataFrame(ttest_results)
print(ttest_df.to_string(index=False))

# Save results
ttest_df.to_csv(FIGURES_PATH + 'ttest_results.csv', index=False)
print("\nSaved: ttest_results.csv")

## Interpretasi T-Test:

**P-Value**:
- **p < 0.05**: Perbedaan signifikan secara statistik - **p ≥ 0.05**: Tidak ada perbedaan signifikan

**Cohen's d (Effect Size)**:
- **|d| < 0.2**: Small effect
- **0.2 ≤ |d| < 0.5**: Medium effect
- **|d| ≥ 0.5**: Large effect
- **|d| ≥ 0.8**: Very large effect

Variabel dengan p-value kecil DAN effect size besar adalah **predictor potensial yang kuat**.

#2.4 BIVARIATE ANALYSIS - Categorical vs Target

Analisis hubungan antara variabel kategorikal dengan target:
- Apakah ada kategori tertentu yang lebih cenderung subscribe?
- Chi-square test untuk uji independensi


In [ ]:
# Stacked bar plots: Categorical variables vs Target
n_categorical = len(categorical_cols)
n_rows = (n_categorical + 2) // 3
fig, axes = plt.subplots(n_rows, 3, figsize=(18, n_rows * 4))
axes = axes.ravel()

for idx, col in enumerate(categorical_cols):
    # Create crosstab (frequency table)
    ct = pd.crosstab(df[col], df['y'], normalize='index') * 100

    # Create stacked bar plot
    ct.plot(kind='bar', stacked=True, ax=axes[idx],
            color=['#FF6B6B', '#4ECDC4'], edgecolor='black')

    axes[idx].set_title(f'{col} vs Target (Percentage)', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Percentage (%)')
    axes[idx].legend(title='Subscribed?', labels=['No', 'Yes'])
    axes[idx].tick_params(axis='x', rotation=45)
    axes[idx].grid(True, alpha=0.3, axis='y')

# Remove empty subplots
for idx in range(len(categorical_cols), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.savefig(FIGURES_PATH + '06_categorical_vs_target.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved: 06_categorical_vs_target.png")

In [ ]:
# Chi-square test untuk variabel kategorikal vs target
# H0: Variabel independent dari target (tidak ada hubungan)
# H1: Variabel dependent (ada hubungan)

print("\nCHI-SQUARE TEST RESULTS: Categorical Variables vs Target")
print("="*70)
print("H0: Variabel independent dari target (tidak ada hubungan)")
print("H1: Variabel dependent dari target (ada hubungan)")
print("Significance level (α): 0.05")
print("="*70)

chisq_results = []

for col in categorical_cols:
    # Create contingency table
    contingency_table = pd.crosstab(df[col], df['y'])

    # Perform chi-square test
    chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)

    # Calculate Cramér's V (effect size for chi-square)
    n = contingency_table.sum().sum()
    cramers_v = np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))

    # Interpret results
    is_significant = "Yes" if p_value < 0.05 else "No"

    chisq_results.append({
        'Variable': col,
        'Chi2_Statistic': chi2,
        'P_Value': p_value,
        'Degrees_of_Freedom': dof,
        'Significant': is_significant,
        'Cramers_V': cramers_v
    })

chisq_df = pd.DataFrame(chisq_results)
print(chisq_df.to_string(index=False))

# Save results
chisq_df.to_csv(FIGURES_PATH + 'chisquare_results.csv', index=False)
print("\nSaved: chisquare_results.csv")

## Interpretasi Chi-Square Test:

**P-Value**:
- **p < 0.05**: Ada hubungan signifikan
- **p ≥ 0.05**: Tidak ada hubungan signifikan

**Cramér's V (Effect Size)**:
- **V < 0.1**: Negligible association
- **0.1 ≤ V < 0.3**: Weak association
- **0.3 ≤ V < 0.5**: Moderate association
- **V ≥ 0.5**: Strong association

Variabel dengan p-value kecil DAN Cramér's V besar menunjukkan **hubungan yang kuat** dengan target.

#2.5 MULTIVARIATE ANALYSIS - Correlation Matrix

**Correlation** = ukuran kekuatan hubungan linear antara 2 variabel numerik.

**Correlation Coefficient (r)**:
- **r = 1**: Perfect positive correlation
- **r = -1**: Perfect negative correlation
- **r = 0**: No linear correlation
- **|r| > 0.7**: Strong correlation
- **|r| > 0.5**: Moderate correlation
- **|r| > 0.3**: Weak correlation

In [ ]:
# Correlation matrix untuk variabel numerik
corr_matrix = df[numerical_cols].corr()

# Create heatmap
plt.figure(figsize=(14, 12))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))  # Mask upper triangle

sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f',
            cmap='coolwarm', center=0, square=True,
            linewidths=1, cbar_kws={"shrink": 0.8})

plt.title('Correlation Matrix - Numerical Variables', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig(FIGURES_PATH + '07_correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved: 07_correlation_matrix.png")

In [ ]:
# Identifikasi highly correlated pairs (potential multicollinearity)
print("\n🔍 HIGHLY CORRELATED VARIABLE PAIRS:")
print("="*70)
print("(|r| > 0.7)")
print("-"*70)

high_corr_pairs = []

for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        corr_value = corr_matrix.iloc[i, j]
        if abs(corr_value) > 0.7:
            high_corr_pairs.append({
                'Variable_1': corr_matrix.columns[i],
                'Variable_2': corr_matrix.columns[j],
                'Correlation': corr_value
            })

if high_corr_pairs:
    high_corr_df = pd.DataFrame(high_corr_pairs).sort_values('Correlation', ascending=False)
    print(high_corr_df.to_string(index=False))
    high_corr_df.to_csv(FIGURES_PATH +'high_correlation_pairs.csv', index=False)
    print("\nSaved: high_correlation_pairs.csv")
else:
    print("Tidak ada pasangan variabel dengan korelasi tinggi (|r| > 0.7)")

##💡 Key Insight #2: Multicollinearity

**Multicollinearity** terjadi ketika variabel predictor sangat berkorelasi satu sama lain.

**Dampak**:
- Membuat model sulit mengidentifikasi efek individual setiap variabel
- Coefficient estimates menjadi tidak stabil
- Standard errors meningkat

**Solusi**:
1. Remove salah satu variabel dari pasangan yang highly correlated
2. Combine kedua variabel (feature engineering)
3. Use dimensionality reduction (PCA)
4. Use regularization (Lasso, Ridge)

In [ ]:
# Scatter plot matrix untuk variabel dengan correlation tinggi (sampling untuk efisiensi)
# Pilih subset variabel yang menarik
interesting_vars = ['age', 'duration', 'campaign', 'previous', 'euribor3m', 'nr.employed']
sample_size = min(1000, len(df))  # Sample untuk efisiensi plotting

sampled_df = df[interesting_vars].sample(n=sample_size, random_state=42)

# Create pair plot
pairplot = sns.pairplot(sampled_df, diag_kind='kde', plot_kws={'alpha': 0.6, 's': 20})
pairplot.fig.suptitle('Pair Plot - Selected Numerical Variables (Sample)',
                       y=1.01, fontsize=16, fontweight='bold')

plt.tight_layout()
plt.savefig(FIGURES_PATH +'08_pairplot_selected_vars.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved: 08_pairplot_selected_vars.png")

---
🎨 PERTEMUAN 3: INSIGHTS & PREPARATION
---


# 3.1 FEATURE ENGINEERING

**Feature Engineering** = proses membuat fitur baru dari fitur yang sudah ada untuk meningkatkan performa model.

Kita akan buat beberapa features baru berdasarkan domain knowledge dan EDA findings.

In [ ]:
# 1. Age groups (demographic segmentation)
df['age_group'] = pd.cut(df['age'],
                         bins=[0, 25, 35, 45, 55, 65, 100],
                         labels=['<25', '25-35', '36-45', '46-55', '56-65', '65+'])

print("Feature created: age_group")
print(df['age_group'].value_counts().sort_index())

In [ ]:
# 2. Contact frequency category (effort dalam kampanye)
df['campaign_intensity'] = pd.cut(df['campaign'],
                                   bins=[0, 1, 3, 5, 100],
                                   labels=['Low (1)', 'Medium (2-3)', 'High (4-5)', 'Very High (6+)'])

print("\nFeature created: campaign_intensity")
print(df['campaign_intensity'].value_counts().sort_index())


In [ ]:
# 3. Previous contact success rate (bagi yang pernah dikontak)
# Note: hanya applicable untuk yang was_contacted_before = 1
df['prev_success_rate'] = 0.0
mask = df['previous'] > 0
df.loc[mask, 'prev_success_rate'] = (df.loc[mask, 'poutcome'] == 'success').astype(int)

print("\nFeature created: prev_success_rate")
print(f"   Previous contacts with success: {(df['prev_success_rate'] == 1).sum()}")

In [ ]:
# 4. Economic indicator composite (dimensionality reduction manual)
# Normalize dulu, lalu combine
from sklearn.preprocessing import StandardScaler

economic_vars = ['emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
scaler = StandardScaler()
df[economic_vars] = scaler.fit_transform(df[economic_vars])

# Create composite score (weighted average atau simple average)
df['economic_score'] = df[economic_vars].mean(axis=1)

print("\nFeature created: economic_score (composite of economic indicators)")
print(f"   Mean: {df['economic_score'].mean():.2f}, Std: {df['economic_score'].std():.2f}")

In [ ]:
# 5. Contact duration categories (proxy untuk interest level)
df['duration_category'] = pd.cut(df['duration'],
                                 bins=[0, 60, 180, 300, 1000000],
                                 labels=['Very Short (<1min)', 'Short (1-3min)',
                                        'Medium (3-5min)', 'Long (>5min)'])

print("\nFeature created: duration_category")
print(df['duration_category'].value_counts().sort_index())

##IMPORTANT NOTE: Duration Variable

Variabel `duration` sangat predictive (call duration panjang = likely subscribe).

**TAPI**: Duration hanya diketahui SETELAH call selesai!

**Untuk Realistic Predictive Model**:
- Duration TIDAK boleh digunakan sebagai predictor
- Kita hanya pakai untuk EDA dan understanding
- Nanti di modeling, kita akan exclude duration

Ini adalah contoh **data leakage** yang harus dihindari.

In [ ]:
# 6. Season/Quarter dari month
season_map = {
    'mar': 'Q1', 'apr': 'Q1', 'may': 'Q1',
    'jun': 'Q2', 'jul': 'Q2', 'aug': 'Q2',
    'sep': 'Q3', 'oct': 'Q3', 'nov': 'Q3',
    'dec': 'Q4', 'jan': 'Q4', 'feb': 'Q4'
}
df['quarter'] = df['month'].map(season_map)

print("\nFeature created: quarter")
print(df['quarter'].value_counts().sort_index())

In [ ]:
# 7. Binary flags untuk interpretability
df['has_default'] = (df['default'] == 'yes').astype(int)
df['has_housing_loan'] = (df['housing'] == 'yes').astype(int)
df['has_personal_loan'] = (df['loan'] == 'yes').astype(int)
df['is_married'] = (df['marital'] == 'married').astype(int)

print("\nBinary features created:")
print(f"   - has_default: {df['has_default'].sum()} ({100*df['has_default'].mean():.2f}%)")
print(f"   - has_housing_loan: {df['has_housing_loan'].sum()} ({100*df['has_housing_loan'].mean():.2f}%)")
print(f"   - has_personal_loan: {df['has_personal_loan'].sum()} ({100*df['has_personal_loan'].mean():.2f}%)")
print(f"   - is_married: {df['is_married'].sum()} ({100*df['is_married'].mean():.2f}%)")

#3.2 VISUAL STORYTELLING

Sekarang kita buat visualisasi yang **menceritakan story** berdasarkan findings kita.

In [ ]:
# Story 1: Age & Job profile dari subscribers vs non-subscribers
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Age distribution by target
for target_val in ['no', 'yes']:
    subset = df[df['y'] == target_val]['age']
    axes[0, 0].hist(subset, bins=30, alpha=0.6, label=f'Subscribed: {target_val}', edgecolor='black')

axes[0, 0].set_title('Age Distribution by Subscription Status', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Age')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Age group success rate
age_group_rate = df.groupby('age_group')['y'].apply(lambda x: (x == 'yes').mean() * 100).sort_index()
age_group_rate.plot(kind='bar', ax=axes[0, 1], color='teal', edgecolor='black')
axes[0, 1].set_title('Subscription Rate by Age Group', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Age Group')
axes[0, 1].set_ylabel('Subscription Rate (%)')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3, axis='y')

for i, v in enumerate(age_group_rate.values):
    axes[0, 1].text(i, v + 0.5, f'{v:.1f}%', ha='center', va='bottom', fontweight='bold')

# 3. Top jobs by volume
top_jobs = df['job'].value_counts().head(10)
top_jobs.plot(kind='barh', ax=axes[1, 0], color='steelblue', edgecolor='black')
axes[1, 0].set_title('Top 10 Jobs by Volume', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Count')
axes[1, 0].set_ylabel('Job Type')
axes[1, 0].grid(True, alpha=0.3, axis='x')

# 4. Job subscription rate (top 10)
job_rate = df.groupby('job')['y'].apply(lambda x: (x == 'yes').mean() * 100)
job_rate_top = job_rate.sort_values(ascending=False).head(10)
job_rate_top.plot(kind='barh', ax=axes[1, 1], color='coral', edgecolor='black')
axes[1, 1].set_title('Top 10 Jobs by Subscription Rate', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Subscription Rate (%)')
axes[1, 1].set_ylabel('Job Type')
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(FIGURES_PATH +'09_story_demographics.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved: 09_story_demographics.png")


In [ ]:
# Story 2: Campaign characteristics & effectiveness
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Campaign intensity vs success rate
campaign_success = df.groupby('campaign_intensity')['y'].apply(lambda x: (x == 'yes').mean() * 100)
campaign_success.plot(kind='bar', ax=axes[0, 0], color='orange', edgecolor='black')
axes[0, 0].set_title('Subscription Rate by Campaign Intensity', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Campaign Intensity')
axes[0, 0].set_ylabel('Subscription Rate (%)')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3, axis='y')

for i, v in enumerate(campaign_success.values):
    axes[0, 0].text(i, v + 0.5, f'{v:.1f}%', ha='center', va='bottom', fontweight='bold')

# 2. Duration vs subscription (box plot)
df.boxplot(column='duration', by='y', ax=axes[0, 1], patch_artist=True)
axes[0, 1].set_title('Call Duration by Subscription Status', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Subscribed?')
axes[0, 1].set_ylabel('Duration (seconds)')
axes[0, 1].get_figure().suptitle('')  # Remove default title

# 3. Month performance
month_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
month_rate = df.groupby('month')['y'].apply(lambda x: (x == 'yes').mean() * 100)
month_rate = month_rate.reindex(month_order, fill_value=0)
month_rate.plot(kind='bar', ax=axes[1, 0], color='purple', edgecolor='black')
axes[1, 0].set_title('Subscription Rate by Month', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Subscription Rate (%)')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3, axis='y')

# 4. Previous outcome impact
poutcome_rate = df.groupby('poutcome')['y'].apply(lambda x: (x == 'yes').mean() * 100)
poutcome_rate.plot(kind='bar', ax=axes[1, 1], color='green', edgecolor='black')
axes[1, 1].set_title('Subscription Rate by Previous Campaign Outcome', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Previous Outcome')
axes[1, 1].set_ylabel('Subscription Rate (%)')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3, axis='y')

for i, v in enumerate(poutcome_rate.values):
    axes[1, 1].text(i, v + 1, f'{v:.1f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig(FIGURES_PATH +'10_story_campaign.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved: 10_story_campaign.png")

In [ ]:
# Story 3: Economic context
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Scatter: euribor vs subscription rate
euribor_bins = pd.cut(df['euribor3m'], bins=20)
euribor_rate = df.groupby(euribor_bins)['y'].apply(lambda x: (x == 'yes').mean() * 100)
euribor_midpoints = [interval.mid for interval in euribor_rate.index]

axes[0, 0].scatter(euribor_midpoints, euribor_rate.values, s=100, alpha=0.6, color='red', edgecolor='black')
axes[0, 0].plot(euribor_midpoints, euribor_rate.values, color='darkred', linewidth=2, alpha=0.7)
axes[0, 0].set_title('Subscription Rate vs Euribor 3M Rate', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Euribor 3M Rate')
axes[0, 0].set_ylabel('Subscription Rate (%)')
axes[0, 0].grid(True, alpha=0.3)

# 2. Employment variation rate
emp_bins = pd.cut(df['emp.var.rate'], bins=10)
emp_rate = df.groupby(emp_bins)['y'].apply(lambda x: (x == 'yes').mean() * 100)
emp_midpoints = [interval.mid for interval in emp_rate.index]

axes[0, 1].bar(range(len(emp_rate)), emp_rate.values, color='skyblue', edgecolor='black')
axes[0, 1].set_title('Subscription Rate by Employment Variation Rate', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Employment Variation Rate (binned)')
axes[0, 1].set_ylabel('Subscription Rate (%)')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# 3. Consumer confidence index
conf_bins = pd.cut(df['cons.conf.idx'], bins=15)
conf_rate = df.groupby(conf_bins)['y'].apply(lambda x: (x == 'yes').mean() * 100)
conf_midpoints = [interval.mid for interval in conf_rate.index]

axes[1, 0].scatter(conf_midpoints, conf_rate.values, s=100, alpha=0.6, color='green', edgecolor='black')
axes[1, 0].plot(conf_midpoints, conf_rate.values, color='darkgreen', linewidth=2, alpha=0.7)
axes[1, 0].set_title('Subscription Rate vs Consumer Confidence Index', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Consumer Confidence Index')
axes[1, 0].set_ylabel('Subscription Rate (%)')
axes[1, 0].grid(True, alpha=0.3)

# 4. Economic score distribution by target
for target_val in ['no', 'yes']:
    subset = df[df['y'] == target_val]['economic_score']
    axes[1, 1].hist(subset, bins=30, alpha=0.6, label=f'Subscribed: {target_val}', edgecolor='black')

axes[1, 1].set_title('Economic Score Distribution by Subscription Status', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Economic Score (Composite)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_PATH +'11_story_economic.png', dpi=300, bbox_inches='tight')
plt.show()

print("Saved: 11_story_economic.png")

#3.3 KEY INSIGHTS & BUSINESS RECOMMENDATIONS

Berdasarkan analisis komprehensif, berikut adalah key insights:

In [ ]:
print("\n" + "="*80)
print("KEY INSIGHTS & FINDINGS")
print("="*80)

insights = """
1. CLASS IMBALANCE
   - Hanya ~11% nasabah yang subscribe term deposit
   - Ini adalah typical challenge dalam marketing campaign
   - Recommendation: Use appropriate evaluation metrics (F1, AUC-ROC, not just accuracy)

2. DEMOGRAPHIC PATTERNS
   - Age: Nasabah berusia 56-65 tahun memiliki conversion rate tertinggi
   - Job: Students dan retired people lebih likely subscribe
   - Education: Nasabah dengan pendidikan tinggi cenderung lebih tertarik
   - Recommendation: Target segmen senior dan high-education background

3. CAMPAIGN CHARACTERISTICS
   - Duration: SANGAT predictive (tapi tidak boleh dipakai untuk prediction!)
   - Campaign intensity: Less is more - terlalu banyak contact menurunkan success rate
   - Previous success: Jika previous campaign success, jauh lebih likely subscribe lagi
   - Recommendation: Focus pada quality over quantity, follow-up previous success

4. TEMPORAL PATTERNS
   - Month: Maret, September, Oktober, Desember memiliki success rate lebih tinggi
   - Day of week: Tidak ada perbedaan signifikan
   - Recommendation: Time campaign di bulan-bulan dengan conversion rate tinggi

5. ECONOMIC INDICATORS
   - Strong correlation antara economic indicators dengan subscription
   - Euribor rate: Negative correlation dengan subscription (higher rate = lower subscription)
   - Employment variation: Negative correlation
   - Recommendation: Consider macroeconomic conditions dalam campaign planning

6. MULTICOLLINEARITY CONCERN
   - Economic indicators sangat correlated satu sama lain
   - Feature engineering (economic_score) dapat help reduce dimensionality
   - Recommendation: Consider regularization atau dimensionality reduction

7. CONTACT METHOD
   - Cellular contact lebih efektif dibanding telephone
   - Recommendation: Prioritize cellular contact method

8. LOAN STATUS
   - Nasabah dengan personal loan atau housing loan slightly less likely subscribe
   - Default history: Very few cases, tapi sangat negative impact
   - Recommendation: Screen for financial stability

KEY VARIABLES FOR MODELING (berdasarkan statistical tests):
   HIGHEST IMPACT:
   - duration (exclude for realistic model!)
   - poutcome (previous campaign outcome)
   - euribor3m (economic indicator)
   - nr.employed (economic indicator)
   - month (temporal)

   MODERATE IMPACT:
   - age, job, education
   - contact method
   - campaign intensity
"""

print(insights)

# Save insights to text file
with open(FIGURES_PATH + 'key_insights.txt', 'w') as f:
    f.write(insights)

print("\nSaved: key_insights.txt")
print("="*80)

## 3.4 DATA PREPARATION FOR MODELING

Sekarang kita prepare data yang clean dan ready untuk modeling.


In [ ]:
# Create final dataset untuk modeling
df_clean = df.copy()

print("\nDATA PREPARATION FOR MODELING:")
print("="*70)

# 1. Encode target variable
df_clean['target'] = (df_clean['y'] == 'yes').astype(int)
print(f"Target encoded: yes=1, no=0")

# 2. Handle unknown values dalam kategorikal
# Strategy: Treat 'unknown' sebagai kategori tersendiri (for now)
# Alternative: Impute dengan mode atau build imputation model
print(f"\nUnknown values kept as separate category (for now)")

# 3. List features untuk modeling (exclude duration untuk realistic model)
features_to_use = [
    # Demographic
    'age', 'job', 'marital', 'education',
    # Financial
    'default', 'housing', 'loan',
    'has_default', 'has_housing_loan', 'has_personal_loan',
    # Campaign
    'contact', 'month', 'day_of_week', 'campaign',
    # Previous campaign
    'pdays', 'previous', 'poutcome', 'was_contacted_before',
    # Economic
    'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
    'economic_score',
    # Engineered features
    'age_group', 'campaign_intensity', 'quarter', 'is_married'
]

print(f"\nFeatures selected for modeling: {len(features_to_use)}")

# 4. Separate features and target
X = df_clean[features_to_use].copy()
y = df_clean['target'].copy()

print(f"\nFeatures (X): {X.shape}")
print(f"Target (y): {y.shape}")

# 5. Identify column types for encoding later
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"\nNumerical features: {len(numerical_features)}")
print(f"Categorical features: {len(categorical_features)}")

##Catatan untuk File Modeling (File 2):

**Yang perlu dilakukan di File 2:**
1. **Encoding categorical variables**:
   - One-Hot Encoding untuk nominal variables
   - Label/Ordinal Encoding untuk ordinal variables
   - Consider target encoding untuk high-cardinality variables

2. **Feature scaling**:
   - StandardScaler atau MinMaxScaler untuk numerical features
   - Fit pada training set, transform pada test set

3. **Train-test split**:
   - Stratified split karena class imbalance
   - Typical ratio: 70-30 atau 80-20

4. **Handle class imbalance**:
   - SMOTE (Synthetic Minority Over-sampling)
   - Class weights dalam model
   - Threshold adjustment

5. **Model selection & evaluation**:
   - Decision Tree dengan GridSearchCV
   - Metrics: F1-score, AUC-ROC, Precision-Recall
   - Feature importance analysis
   - LIME untuk explainability

In [ ]:
# Create comprehensive dataset untuk export
# Include both original and engineered features

# Columns to export
export_columns = features_to_use + ['target', 'duration', 'y']  # Include duration untuk reference
df_export = df_clean[export_columns].copy()

print("\nPREPARING EXPORT:")
print("="*70)
print(f"Dataset shape: {df_export.shape}")
print(f"Features: {len(features_to_use)}")
print(f"Target distribution:")
print(df_export['target'].value_counts())
print(f"\nClass balance: {100*df_export['target'].mean():.2f}% positive class")

In [ ]:
# Export cleaned dataset
df_export.to_csv(BASE_PATH + 'bank_marketing_cleaned.csv', index=False)

print("\n" + "="*70)
print("EXPORT COMPLETED!")
print("="*70)
print(f"📁 File saved: {BASE_PATH}bank_marketing_cleaned.csv")
print(f"Rows: {len(df_export):,}")
print(f"Columns: {len(df_export.columns)}")
print("\nDATASET READY FOR MODELING! 🚀")
print("="*70)

## 3.5 DATA QUALITY REPORT (SUMMARY)

In [ ]:
# Generate comprehensive data quality report
print("\n" + "="*80)
print("FINAL DATA QUALITY REPORT")
print("="*80)

report = f"""
DATASET INFORMATION:
- Original rows: {len(df_original):,}
- Final rows: {len(df_export):,}
- Original columns: {len(df_original.columns)}
- Final columns: {len(df_export.columns)}

MISSING VALUES:
- Original dataset: {df_original.isnull().sum().sum()} missing values
- Final dataset: {df_export.isnull().sum().sum()} missing values

DUPLICATES:
- Duplicate rows removed: 0

DATA TYPES:
- Numerical features: {len(numerical_features)}
- Categorical features: {len(categorical_features)}

TARGET VARIABLE:
- Positive class (y=1): {(df_export['target']==1).sum():,} ({100*df_export['target'].mean():.2f}%)
- Negative class (y=0): {(df_export['target']==0).sum():,} ({100*(1-df_export['target'].mean()):.2f}%)
- Class imbalance ratio: 1:{(df_export['target']==0).sum()/(df_export['target']==1).sum():.1f}

OUTLIERS:
- Variables with outliers: {len(outlier_df[outlier_df['Outliers_Count'].astype(str).str.replace('%','').astype(float) > 0])}
- See outlier_summary.csv for details

FEATURE ENGINEERING:
- Original features: {len(df_original.columns)}
- Engineered features: {len(df_export.columns) - len(df_original.columns)}
- New features: age_group, campaign_intensity, economic_score, duration_category,
                quarter, was_contacted_before, has_default, has_housing_loan,
                has_personal_loan, is_married, target

STATISTICAL TESTS PERFORMED:
- T-tests (numerical vs target): {len(ttest_df)}
- Chi-square tests (categorical vs target): {len(chisq_df)}
- Correlation analysis: Completed

EXPORTED FILES:
1. bank_marketing_cleaned.csv (main dataset for modeling)
2. All figures saved to ./figures/ directory
3. Statistical test results (CSV)
4. Descriptive statistics (CSV)
5. Key insights (TXT)

RECOMMENDATIONS FOR MODELING:
1. Address class imbalance (SMOTE, class weights, or threshold tuning)
2. Exclude 'duration' for realistic predictive model
3. Use stratified train-test split
4. Apply appropriate encoding for categorical variables
5. Scale numerical features
6. Consider regularization due to multicollinearity in economic indicators
7. Use appropriate evaluation metrics (F1, AUC-ROC, Precision-Recall)
8. Perform feature selection based on importance
9. Cross-validation for robust evaluation
10. Document model assumptions and limitations

NEXT STEPS (FILE 2 - MODELING):
→ Load bank_marketing_cleaned.csv
→ Encode categorical variables
→ Scale numerical features
→ Train-test split (stratified)
→ Handle class imbalance
→ Build Decision Tree with GridSearchCV
→ Evaluate with multiple metrics
→ Feature importance analysis
→ Explainability with LIME
→ Model interpretation & business insights
"""

print(report)

# Save report
with open(FIGURES_PATH + 'data_quality_report.txt', 'w') as f:
    f.write(report)

print("\nSaved: data_quality_report.txt")
print("="*80)

---
TUTORIAL COMPLETED!
---

#📚 What You've Learned:

##**Pertemuan 1: Foundation**
Data loading dan initial exploration  
Data quality assessment (missing values, duplicates, data types)  
Data cleaning dan basic transformation  
Descriptive statistics (mean, median, std, quartiles, skewness, kurtosis)  
Understanding target variable distribution  

##**Pertemuan 2: Deep Exploration**
Univariate analysis (distributions, outliers dengan IQR method)  
Bivariate analysis (numerical vs target, categorical vs target)  
Statistical hypothesis testing (t-test, chi-square test)  
Effect size interpretation (Cohen's d, Cramér's V)  
Correlation analysis dan multicollinearity detection  
Multivariate patterns dengan pair plots  

##**Pertemuan 3: Insights & Preparation**
Feature engineering (binning, composite features, binary flags)  
Visual storytelling dengan meaningful plots  
Business insights generation  
Data preparation untuk modeling  
Comprehensive data quality reporting  

#Key Takeaways:

1. **EDA is iterative** - Terus explore sampai punya deep understanding
2. **Statistik ≠ ML** - Statistik untuk understanding, ML untuk prediction
3. **Context matters** - Business knowledge sama pentingnya dengan technical skills
4. **Data quality first** - Garbage in, garbage out
5. **Visual storytelling** - Plot yang baik worth a thousand words
6. **Feature engineering** - Domain knowledge dapat boost model performance significantly
7. **Document everything** - Reproducibility dan transparency are crucial

#📖 Recommended Next Steps:

1. **Practice**: Coba dataset lain dengan domain berbeda
2. **Deepen statistics**: Belajar lebih dalam tentang inferential statistics
3. **Advanced visualization**: Explore plotly, altair untuk interactive viz
4. **Feature selection**: Learn about LASSO, RFE, feature importance
5. **Ready for modeling**: Move to File 2 untuk classification modeling!

#💡 Pro Tips untuk Tugas Akhir:

- **Start with good EDA**: Pahami data dulu sebelum modeling
- **Document as you go**: Jangan tunggu sampai selesai
- **Tell a story**: Connect findings dengan research questions
- **Be critical**: Question assumptions, validate findings
- **Iterate**: EDA bukan one-time process
- **Communicate clearly**: Technical correctness + clear communication = excellent thesis

---

#🚀 READY FOR MODELING!

Dataset sudah clean, features sudah engineered, insights sudah digali.  
**Next**: Build robust classification model dengan Decision Tree, GridSearch, dan LIME!

---

**Good luck dengan pembelajaran dan tugas akhir! 💪**

In [ ]:
# Final check: Display first few rows of cleaned dataset
print("\n" + "="*80)
print("PREVIEW: bank_marketing_cleaned.csv")
print("="*80)
print(df_export.head(10))
print("\n" + "="*80)
print("END OF EDA TUTORIAL")
print("="*80)